In [28]:
library(dplyr)
library(parallel)
library(stringr)

hubname='CU'
continent='na'
lakename='PP'
PT_key_file='SWOTCalVal_PP_KEY_20230515_20230627_1.csv'
utm_zone=14 #WM= 10

# hubname='CU'
# continent='na'
# lakename='PNW'
# #PT_key_file= 'SWOTCalVal_PNW_KEY_20230504.csv' #WM
# utm_zone=10 #WM= 10

# hubname='UNC'
# continent='na'
# lakename='SL'
# #PT_key_file= '' #WM
# utm_zone=11 #WM= 10

# hubname='UNC'
# continent='na'
# lakename='YF'
# #PT_key_file='' #WM
# utm_zone=6 #WM= 10

setwd(paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/'))
raw_GNSS_drift_data_directory=paste0('From Andy/',hubname,'_netCDFs/')
QA_QC_drift_output_directory= '/nas/cee-water/cjgleason/calval/Processed data/CU/Munged_lake_gnss/'
QA_QC_PT_output_directory= '/nas/cee-water/cjgleason/calval/Processed data/CU/Munged_lake_PT/'
flagged_drift_output_directory='/nas/cee-water/cjgleason/calval/Processed data/CU/flagged_lake_drifts/'
flagged_PT_output_directory='/nas/cee-water/cjgleason/calval/Processed data/CU/flagged_lake_PT/'
raw_PT_data_directory= '/nas/cee-water/cjgleason/calval/xml_scripts/CU/Munged/'


time_thresh=15*60
dist_thresh=1000

In [26]:
#create dataframes from drifts---------------------------------------------------------
#pull filename before the .csv
source('/nas/cee-water/cjgleason/calval_toolbox/R code/create_lake_GNSS_dataframe.R')
raw_GNSS=sub( "\\..*","", list.files(raw_GNSS_drift_data_directory,recursive=TRUE))
raw_GNSS_lake=which(!is.na(do.call(rbind,lapply(raw_GNSS,str_match,lakename))))
raw_GNSS=raw_GNSS[raw_GNSS_lake]

#pull filename before the second _
QA_QC_drifts=sub( "\\..*","",list.files(QA_QC_drift_output_directory))
flagged_drifts=sub("\\..*","",list.files(flagged_drift_output_directory))
#what raw drift data have not been munged
unmunged_drifts=setdiff(raw_GNSS,c(flagged_drifts,QA_QC_drifts))
# print(raw_GNSS)
print(unmunged_drifts)

# create_lake_gnss_dataframe(unmunged_drifts[1],
#                   gnss_drift_data_directory=GNSS_drift_data_directory,
#                   output_directory=QA_QC_drift_output_directory)


cl=makeCluster(44)

  dummy=parLapply(cl=cl,unmunged_drifts,create_lake_gnss_dataframe,
                  gnss_drift_data_directory=raw_GNSS_drift_data_directory,
                  output_directory=QA_QC_drift_output_directory)
stopCluster(cl)



[1] "SWOTCalVal_PP_GNSS_L2_Rec3_20230627T175730_20230627T185745_20230724T222104"
[2] "SWOTCalVal_PP_GNSS_L2_Rec3_20230627T202900_20230627T213010_20230724T222731"


In [30]:
#correct PTs to the GNSS from the key to the lake
source('/nas/cee-water/cjgleason/calval_toolbox/R code/lake_PT.r')

GNSS_sd_thresh=0.15 # 15cm how much variance do you want in the GNSS data when it is within the distance threshold?
offset_sd_thresh=0.10 #m, so 10cm. the the PT apparantly shift by more than a cm?
change_thresh_15_min=0.05#m- does it change more than 5cm in 15 minutes? that is a discontinuity in offset


#get all the PTs in the data directory
munged_files= list.files(raw_PT_data_directory,recursive= TRUE)

PT_index=which(!is.na(do.call(rbind,lapply(munged_files,str_match,'PT_L1'))))
PT_files=munged_files[PT_index]

csv_index=which(!is.na(do.call(rbind,lapply(PT_files,str_match,'.csv'))))
csv_PT_files=PT_files[csv_index]

correct_lake_index=which(!is.na(do.call(rbind,lapply(csv_PT_files,str_match,lakename))))
csv_PT_files=csv_PT_files[correct_lake_index]

# print(correct_lake_index)

#pull filename before the .csv
raw_PT=sub("\\..*","",csv_PT_files)
raw_PT=sub(".*/","",raw_PT)
#pull filenam\\..*e before the second _
QA_QC_PTs=sub("\\..*","",list.files(QA_QC_PT_output_directory))

#what raw PT data have not been munged
    #need to preserve the full path in csv_PT_files!
unmunged_PTs=setdiff(csv_PT_files,QA_QC_PTs)
print(unmunged_PTs)

# test=lake_PT(unmunged_PTs[1],
#              pt_key_file=PT_key_file,
#              pt_data_directory=raw_PT_data_directory,
#              dist_thresh=dist_thresh,
#              time_thresh=time_thresh,
#              gnss_drift_data_directory=QA_QC_drift_output_directory,
#              QA_QC_pt_output_directory=QA_QC_PT_output_directory,
#              flagged_pt_output_directory=flagged_PT_output_directory,
#              gnss_sd_thresh=GNSS_sd_thresh,
#              offset_sd_thresh=offset_sd_thresh,
#              change_thresh_15_min=change_thresh_15_min)



    cl=makeCluster(44,type='FORK')
  
  dummy=parLapply(cl, unmunged_PTs,lake_PT,
             pt_key_file=PT_key_file,
             pt_data_directory=raw_PT_data_directory,
             dist_thresh=dist_thresh,
             time_thresh=time_thresh,
             gnss_drift_data_directory=QA_QC_drift_output_directory,
             QA_QC_pt_output_directory=QA_QC_PT_output_directory,
             flagged_pt_output_directory=flagged_PT_output_directory,
             gnss_sd_thresh=GNSS_sd_thresh,
             offset_sd_thresh=offset_sd_thresh,
             change_thresh_15_min=change_thresh_15_min)
  
  stopCluster(cl)




 [1] "Munged__20230725/SWOTCalVal_PP_PT_L1_PT170_20230515T140000_20230626T154500_20230725T212325.csv"
 [2] "Munged__20230725/SWOTCalVal_PP_PT_L1_PT171_20230515T140000_20230626T160000_20230725T212325.csv"
 [3] "Munged__20230725/SWOTCalVal_PP_PT_L1_PT172_20230515T140000_20230626T180000_20230725T212326.csv"
 [4] "Munged__20230725/SWOTCalVal_PP_PT_L1_PT173_20230515T140000_20230626T194500_20230725T212326.csv"
 [5] "Munged__20230725/SWOTCalVal_PP_PT_L1_PT174_20230515T140000_20230626T213000_20230725T212327.csv"
 [6] "Munged__20230725/SWOTCalVal_PP_PT_L1_PT175_20230515T140000_20230627T154500_20230725T212327.csv"
 [7] "Munged__20230725/SWOTCalVal_PP_PT_L1_PT176_20230515T140000_20230627T140000_20230725T212328.csv"
 [8] "Munged__20230725/SWOTCalVal_PP_PT_L1_PT177_20230515T140000_20230626T224500_20230725T212329.csv"
 [9] "Munged__20230725/SWOTCalVal_PP_PT_L1_PT178_20230515T140000_20230627T170000_20230725T212330.csv"
[10] "Munged__20230725/SWOTCalVal_PP_PT_L1_PT179_20230515T140000_20230627T193000_2